<a href="https://colab.research.google.com/github/emachernova/fcs_llm_2025/blob/main/Homework_2_4_chernova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers faiss-cpu
!pip install datasets pandas numpy torch
!pip install openai tiktoken langchain
!pip install chromadb # альтернатива FAISS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.1 MB/s eta 0

In [ ]:
from datasets import load_dataset
import pandas as pd
import json
from typing import List, Dict, Tuple
import hashlib

# Загружаем SQuAD 2.0 (Stanford Question Answering Dataset)
dataset = load_dataset("squad_v2", split="train[:1000]")  # Берем первые 1000 примеров

class SQuADProcessor:
    """Обработчик датасета SQuAD для RAG"""
    # извлекает уникальные контексты (параграфы)
    # создает список QA-пар (вопрос + ответ + ссылка на контекст)

    def __init__(self):
        self.documents = {}
        self.qa_pairs = []

    def process_squad(self, dataset) -> Tuple[List[str], List[Dict]]:
        """
        Извлекает уникальные контексты и QA пары из SQuAD
        """
        contexts_dict = {}
        qa_pairs = []

        for item in dataset:
            context = item['context']
            context_hash = hashlib.md5(context.encode()).hexdigest()[:8]

            # берем первые 8 символов хэша MD5 от текста - чтобы создать короткий, уникальный идентификатор для каждого документа

            # Сохраняем уникальные контексты
            if context_hash not in contexts_dict:
                contexts_dict[context_hash] = {
                    'text': context,
                    'title': item['title'],
                    'id': context_hash
                }

            # Сохраняем QA пары для валидации
            if len(item['answers']['text']) > 0:  # Только вопросы с ответами
                qa_pairs.append({
                    'question': item['question'],
                    'answer': item['answers']['text'][0] if item['answers']['text'] else None,
                    'context_id': context_hash,
                    'is_impossible': item.get('is_impossible', False)
                })

        documents = list(contexts_dict.values())
        print(f"Обработано {len(documents)} уникальных документов")
        print(f"Обработано {len(qa_pairs)} QA пар")

        return documents, qa_pairs

        # список используется для построения векторного индекса (Retriever)
        # тестирование качества RAG (модель должна найти контекст и сгенерировать правильный ответ)

# Инициализация и обработка
processor = SQuADProcessor()
documents, qa_pairs = processor.process_squad(dataset)

# Пример документа
print("Пример документа:")
print(f"Title: {documents[0]['title']}")
print(f"Text (первые 200 символов): {documents[0]['text'][:200]}...")
print(f"\nПример QA пары:")
print(f"Q: {qa_pairs[0]['question']}")
print(f"A: {qa_pairs[0]['answer']}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Обработано 85 уникальных документов
Обработано 1000 QA пар
Пример документа:
Title: Beyoncé
Text (первые 200 символов): Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in v...

Пример QA пары:
Q: When did Beyonce start becoming popular?
A: in the late 1990s




# 🔑 API ключ для учебных целей

Если у вас нет ключа от какого-нибудь API, то можете воспользоваться вот этим ключом:   
sk-f552fcc16e4f419ba8b780cd4882a7d9

## Рекомендации по использованию

* Используйте ключ **только для домашних заданий**
* Помните, что им пользуются и другие студенты - будьте аккуратны с количеством запросов


In [ ]:
import os
from getpass import getpass
os.environ["API_KEY"] = getpass("Введите ключ : ")

Введите ключ : ··········


In [ ]:
import os
from dataclasses import dataclass
from typing import Optional

@dataclass
class DeepSeekRAGConfig:
    """Конфигурация для RAG с DeepSeek"""

    # DeepSeek API настройки
    api_key=os.environ.get("API_KEY")
    deepseek_api_key: str = api_key
    deepseek_base_url: str = "https://api.deepseek.com/v1"
    deepseek_model: str = "deepseek-chat"

    # Embedding модель
    embedding_model: str = "BAAI/bge-small-en-v1.5"  # Компактная и эффективная
    embedding_dim: int = 384 # размер эмбеддинга

    # Chunking параметры (разбиение текста на куски)
    chunk_size: int = 300  # Меньше чем обычно для SQuAD (короткие параграфы)
    chunk_overlap: int = 50 # нужен, чтобы не терять смысл на стыках

    # Retrieval параметры (поиск контента)
    retrieval_top_k: int = 5 # сначала извлекаем 5 наиболее похожих фрагмента
    rerank_top_k: int = 3 # потом можем отобрать из них 3 лучших после переранжировки

    # Generation параметры (генерация ответа)
    max_tokens: int = 150 # ограничивает длину ответа
    temperature: float = 0.1  # Низкая для фактической точности

    # System design
    cache_enabled: bool = True
    batch_size: int = 32

config = DeepSeekRAGConfig()


Задание 1.    
text_chunks =  self.splitter.split_text(cleaned_text)

normalized_text = re.sub(r"\s+", " ", raw_text)   

length_chars = len(chunk_text)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

class SmartDocumentChunker:
    """Интеллектуальный чанкинг для работы с SQuAD документами"""

    def __init__(self, rag_config: DeepSeekRAGConfig):
        self.config = rag_config
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=rag_config.chunk_size,
            chunk_overlap=rag_config.chunk_overlap,
            separators=["\n\n", "\n", ". ", ", ", " "],
            length_function=len,
        )

    def process_documents(self, docs_list: List[Dict]) -> List[Dict]:
        """Обрабатывает документы и создает чанки с метаинформацией"""
        chunks_collection = []

        for document in docs_list:
            # Очистка и подготовка текста
            cleaned_text = self._clean_text(document['text'])

            # TODO: Разделите cleaned_text на части используя self.splitter
            # Подсказка: используйте метод split_text()
            text_chunks = self.splitter.split_text(cleaned_text)

            # делим по абзацам, потом по строкам, потом по предложениям, далее по словам

            # Формирование чанков с метаданными
            for idx, chunk_text in enumerate(text_chunks):
                chunk_info = {
                    'chunk_id': f"{document['id']}_part_{idx}",
                    'text_content': chunk_text,
                    'meta_info': {
                        'source_doc_id': document['id'],
                        'doc_title': document['title'],
                        'position': idx,
                        'chunks_total': len(text_chunks),
                        # TODO: Вычислите длину текста в символах
                        'length_chars': len(chunk_text),
                        'words_count': len(chunk_text.split())
                    }
                }
                chunks_collection.append(chunk_info)

        print(f"Обработано: {len(chunks_collection)} чанков из {len(docs_list)} документов")
        return chunks_collection

    def _clean_text(self, raw_text: str) -> str:
        """Очищает и нормализует текст перед разбиением"""
        # TODO: Замените множественные пробелы на один пробел
        # Подсказка: используйте re.sub() с паттерном \s+
        normalized_text = re.sub(r"\s+", " ", raw_text)

        # Добавляем точку если текст не заканчивается знаком препинания
        if normalized_text and normalized_text[-1] not in '.!?':
            normalized_text += '.'
        return normalized_text.strip()

# Создаем и применяем чанкер
doc_chunker = SmartDocumentChunker(config)
processed_chunks = doc_chunker.process_documents(documents)
print(f"Пример обработанного чанка:\n{processed_chunks[0]['text_content'][:150]}...")


Обработано: 334 чанков из 85 документов
Пример обработанного чанка:
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress...


Комментарии (для меня):



*   берем длинный текст (wikipedia-контекст из SQuAD)
*   чистим его от лишних пробелов и обрезков форматирования
*   дробим на куски (чанки) разумного размера - чтобы embedding-модель могла их векторизовать

*   каждый чнк получает метаданные, чтобы потом можно было восстановить, из какого документа он был взят
*   всё это потом пойдет в векторное хранилище









Задание 2   
1- texts = [chunk['text_content'] for chunk in chunks]  
2- embeddings = self.embedder.encode(
            texts,
            batch_size = self.config.batch_size,
            show_progress_bar=True,
            normalize_embeddings=True
        )  
3 - def _calculate_relevance:
        return return (distance + 1)/2

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.4 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from tqdm import tqdm

class OptimizedVectorStore:
    """Оптимизированное векторное хранилище с FAISS"""

    def __init__(self, config: DeepSeekRAGConfig):
        self.config = config
        self.embedder = SentenceTransformer(config.embedding_model)

        # Инициализация FAISS индекса с IVF для масштабируемости
        self.index = None
        self.metadata = {}
        self.embeddings_cache = {}

    def build_index(self, chunks: List[Dict], use_gpu: bool = False):
        """Строит векторный индекс из чанков"""

        print("Генерация эмбеддингов...")
        # ЗАДАНИЕ 1: Извлеките тексты из чанков
        texts = [chunk['text_content'] for chunk in chunks]

        # FAISS работает только с числовыми векторами, поэтому сначала нужен список строк - текстов, которые потом конвертируются в эмбеддинги

        # Batch encoding для эффективности

        # ЗАДАНИЕ 2: реализовать batch encoding текстов
        # используйте self.embedding_model.encode() с правильными параметрами
        # - batch_size из конфигурации
        # - нормализация векторов
        # - progress bar
        embeddings = self.embedder.encode(
            texts,
            batch_size = self.config.batch_size,
            show_progress_bar=True,
            normalize_embeddings=True
        )

        print("Построение FAISS индекса...")
        dimension = embeddings.shape[1]

        # Используем IVF индекс для больших датасетов
        if len(embeddings) > 10000:
            nlist = int(np.sqrt(len(embeddings)))  # Количество кластеров
            quantizer = faiss.IndexFlatL2(dimension)
            self.index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
            self.index.train(embeddings.astype('float32'))
        else:
            # Для небольших датасетов используем Flat индекс
            self.index = faiss.IndexFlatIP(dimension)  # Inner product для косинусного сходства

        # Добавляем векторы в индекс
        self.index.add(embeddings.astype('float32'))

        # Сохраняем метаданные
        for i, chunk in enumerate(chunks):
            self.metadata[i] = chunk

        print(f"Индекс построен: {self.index.ntotal} векторов")

    def search(self, query: str, k: int = 5) -> List[Dict]:
        """Поиск релевантных чанков"""

        # Кэширование запросов
        if self.config.cache_enabled and query in self.embeddings_cache:
            query_embedding = self.embeddings_cache[query]
        else:
            query_embedding = self.embedder.encode(
                [query],
                normalize_embeddings=True
            ).astype('float32')

            if self.config.cache_enabled:
                self.embeddings_cache[query] = query_embedding

        # Поиск в индексе
        distances, indices = self.index.search(query_embedding, k)

        # Формирование результатов
        results = []
        for dist, idx in zip(distances[0], indices[0]):
            if idx != -1 and idx in self.metadata:
                chunk_data = self.metadata[idx].copy()
                chunk_data['score'] = float(dist)  # Косинусное сходство
                chunk_data['relevance'] = self._calculate_relevance(dist)
                results.append(chunk_data)

        return results

    def _calculate_relevance(self, distance: float) -> float:
        """Преобразует distance в relevance score [0, 1]"""
        # ЗАДАНИЕ 3: Нормализация relevance score
        # TODO: Преобразуйте distance в значение от 0 до 1
        # Подсказка: для Inner Product нормализованных векторов,
        # distance уже представляет косинусное сходство

        return (distance + 1)/2

    def save_index(self, path: str):
        """Сохранение индекса на диск"""
        faiss.write_index(self.index, f"{path}.index")
        with open(f"{path}.metadata", 'wb') as f:
            pickle.dump(self.metadata, f)

    def load_index(self, path: str):
        """Загрузка индекса с диска"""
        self.index = faiss.read_index(f"{path}.index")
        with open(f"{path}.metadata", 'rb') as f:
            self.metadata = pickle.load(f)

# Построение индекса
vector_store = OptimizedVectorStore(config)
vector_store.build_index(processed_chunks)

# Тестовый поиск
test_query = "What is the capital of France?"
results = vector_store.search(test_query, k=3)
print(f"\nРезультаты поиска для: '{test_query}'")
for i, result in enumerate(results, 1):
    print(f"{i}. Score: {result['relevance']:.3f}, Text: {result['text_content'][:100]}...")


Генерация эмбеддингов...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Построение FAISS индекса...
Индекс построен: 334 векторов

Результаты поиска для: 'What is the capital of France?'
1. Score: 0.776, Text: Chopin arrived in Paris in late September 1831; he would never return to Poland, thus becoming one o...
2. Score: 0.764, Text: . In 1835 he obtained French citizenship. After a failed engagement to Maria Wodzińska, from 1837 to...
3. Score: 0.756, Text: . Chopin, now alone in Vienna, was nostalgic for his homeland, and wrote to a friend, "I curse the m...


Задание 3.1 промт =    
Задание 3.2 промт =    
Задание 4 "response_format": ...   
Задание 5 функция def _postprocess_answer   

In [ ]:
import requests
import json
from typing import List, Dict, Optional
import time

class DeepSeekGenerator:
    """Генератор ответов с использованием DeepSeek API"""

    def __init__(self, config: DeepSeekRAGConfig):
        self.config = config
        self.api_key = config.deepseek_api_key
        self.base_url = config.deepseek_base_url
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

    def generate(self, query: str, contexts: List[Dict],
                 use_cot: bool = True) -> Dict:
        """
        Генерация ответа с использованием DeepSeek

        Args:
            query: Вопрос пользователя
            contexts: Релевантные контексты из retrieval
            use_cot: Использовать Chain-of-Thought reasoning
        """

        # Подготовка контекста
        context_text = self._format_contexts(contexts)

        # Создание промпта с учетом особенностей DeepSeek
        if use_cot:
            prompt = self._create_cot_prompt(query, context_text)
        else:
            prompt = self._create_standard_prompt(query, context_text)

        # Запрос к DeepSeek API
        try:
            response = self._call_deepseek_api(prompt)
            print("response: ",response)
            # Постобработка ответа
            answer = self._postprocess_answer(response)

            return {
                'answer': answer,
                'raw_response': response,
                'contexts_used': len(contexts),
                'prompt_tokens': self._estimate_tokens(prompt),
                'model': self.config.deepseek_model
            }

        except Exception as e:
            print(f"Ошибка при вызове DeepSeek API: {e}")
            return {
                'answer': f"Извините, произошла ошибка при генерации ответа. {str(e)}",
                'error': str(e)
            }

    def _format_contexts(self, contexts: List[Dict]) -> str:
        """Форматирование контекстов для промпта"""
        formatted_contexts = []

        for i, ctx in enumerate(contexts, 1):
            relevance = ctx.get('relevance', 0)
            content = ctx.get('text_content', ctx.get('content', ''))
            title = ctx.get('meta_info', {}).get('doc_title', 'Unknown')

            # Добавляем маркеры релевантности
            relevance_marker = "⭐" * min(int(relevance * 5), 5)

            formatted_contexts.append(
                f"[Источник {i}] {relevance_marker}\n"
                f"Документ: {title}\n"
                f"Содержание: {content}\n"
            )

        return "\n".join(formatted_contexts)

    def _create_cot_prompt(self, query: str, context: str) -> str:
        """Создание Chain-of-Thought промпта для DeepSeek"""

        json_format = '{"reasoning": "твои размышления", "answer": "финальный ответ"}'

        return f"""You are an intelligent assistant specializing in question answering based on provided context.

            ## Task
            Answer the user's question based ONLY on the provided context, and respond in JSON format. Use step-by-step reasoning.

            ## Context
            {context}

            ## Question
            {query}

            ## Instructions
            1. First, identify the key information needed to answer the question
            2. Then, locate relevant facts in the provided context
            3. Finally, synthesize a clear, accurate answer
            4. If the context doesn't contain sufficient information, clearly state that

            ## Step-by-Step Reasoning
            Let me analyze this step by step and make desicsion by analize.

            Respond ONLY in JSON format as:
            {{
              "reasoning": "step-by-step reasoning:,
              "answer": "final concise answer"
            }}

            """
            # Задание 3.1 Напишите промт так, чтобы на выходе был json и определенные поля для парсинга

    def _create_standard_prompt(self, query: str, context: str) -> str:
        """Создание стандартного промпта"""

        return f"""You are a helpful assistant. Answer the question based only on the provided context.

                Context:
                {context}

                Question:
                {query}

                Instructions:
                - Use ONLY information from the context
                - Be concise and accurate
                - Cite source numbers [1], [2], etc.
                - If information is insufficient, say so clearly

                Respond ONLY in JSON format as:
                {{
                  "reasoning": "brief explanation",
                  "answer": "concise factual answer"
                }}

                """


    def _call_deepseek_api(self, prompt: str) -> str:
        """Вызов DeepSeek API"""

        payload = {
            "model": self.config.deepseek_model,
            "messages": [
                {
                    "role": "system",
                    "content": "You are a precise question-answering assistant. Always base your answers strictly on the provided context."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "temperature": self.config.temperature,
            "max_tokens": self.config.max_tokens,
            "top_p": 0.95,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            "response_format": {"type": "json_object"},
            "stop": None
        }

        # Retry logic для надежности
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.post(
                    f"{self.base_url}/chat/completions",
                    headers=self.headers,
                    json=payload,
                    timeout=30
                )

                if response.status_code == 200:
                    result = response.json()
                    return result['choices'][0]['message']['content']

                elif response.status_code == 429:  # Rate limit
                    wait_time = (attempt + 1) * 2
                    print(f"Rate limit достигнут, ожидание {wait_time} секунд...")
                    time.sleep(wait_time)

                else:
                    print(f"Ошибка API: {response.status_code} - {response.text}")

            except requests.exceptions.RequestException as e:
                print(f"Ошибка запроса (попытка {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    time.sleep(2)

        raise Exception("Не удалось получить ответ от DeepSeek API")

    def _postprocess_answer(self, answer: str) -> str:
        """Постобработка JSON-ответа модели"""

        # Задание 5 напишите код парсинга ответа модели
        try:
            parsed = json.loads(answer)
            return parsed.get("answer", "").strip()
        except json.JSONDecodeError:

            # Если модель вернула невалидный JSON — пробуем эвристически
            match = re.search(r'"answer"\s*:\s*"([^"]+)"', answer)
            if match:
                return match.group(1).strip()

            return answer.strip()


    def _estimate_tokens(self, text: str) -> int:
        """Примерная оценка количества токенов"""
        # Грубая оценка: 1 токен ≈ 4 символа
        return len(text) // 4

# Инициализация генератора
generator = DeepSeekGenerator(config)


Задание 6   
Код сравнения

In [ ]:
class DeepSeekRAG:
    """Полный RAG pipeline с DeepSeek"""

    def __init__(self, config: DeepSeekRAGConfig):
        self.config = config
        self.chunker = SmartDocumentChunker(config)
        self.vector_store = OptimizedVectorStore(config)
        self.generator = DeepSeekGenerator(config)

        # Метрики и кэш
        self.query_cache = {}
        self.metrics = {
            'total_queries': 0,
            'cache_hits': 0,
            'avg_retrieval_score': [],
            'avg_response_time': []
        }

    # 1. Индексация документов
    def index_documents(self, documents: List[Dict]):
        """Индексация документов"""
        print("🔄 Начало индексации документов...")

        # Чанкинг
        self.chunks = self.chunker.process_documents(documents)

        # Построение векторного индекса
        self.vector_store.build_index(self.chunks)

        print(f"Индексация завершена: {len(self.chunks)} чанков")

    # 2. Основной метод ответа
    def query(self, question: str,
              use_cache: bool = True,
              use_cot: bool = False,
              verbose: bool = False) -> Dict:
        """
        Основной метод для ответа на вопросы

        Args:
            question: Вопрос пользователя
            use_cache: Использовать кэширование
            use_cot: Использовать Chain-of-Thought
            verbose: Выводить детальную информацию
        """

        start_time = time.time()
        self.metrics['total_queries'] += 1

        # Проверка кэша
        if use_cache and question in self.query_cache:
            self.metrics['cache_hits'] += 1
            if verbose:
                print("📎 Ответ взят из кэша")
            return self.query_cache[question]

        # 1. Retrieval
        if verbose:
            print(f"🔍 Поиск релевантных документов для: '{question}'")

        retrieval_start = time.time()
        relevant_contexts = self.vector_store.search(
            question,
            k=self.config.retrieval_top_k
        )
        retrieval_time = time.time() - retrieval_start

        if verbose:
            print(f"📚 Найдено {len(relevant_contexts)} релевантных контекстов")
            print(f"📚 Relevant_contexts {relevant_contexts[0]}")
            print(f"   Время поиска: {retrieval_time:.2f} сек")

        # Расчет средней релевантности
        avg_relevance = np.mean([ctx['relevance'] for ctx in relevant_contexts])
        self.metrics['avg_retrieval_score'].append(avg_relevance)

        # 2. Generation
        if verbose:
            print(f"🤖 Генерация ответа с помощью {self.config.deepseek_model}")

        generation_start = time.time()
        generation_result = self.generator.generate(
            question,
            relevant_contexts,
            use_cot=use_cot
        )
        generation_time = time.time() - generation_start

        # Формирование результата
        total_time = time.time() - start_time
        self.metrics['avg_response_time'].append(total_time)

        result = {
            'question': question,
            'answer': generation_result['answer'],
            'contexts': [
                {
                    'content': ctx.get('text_content', ctx.get('content', '')),
                    'relevance': ctx.get('relevance', 0),
                    'metadata': ctx.get('meta_info', ctx.get('metadata', {}))
                }
                for ctx in relevant_contexts
            ],
            'metrics': {
                'retrieval_time': retrieval_time,
                'generation_time': generation_time,
                'total_time': total_time,
                'avg_relevance': avg_relevance,
                'contexts_used': len(relevant_contexts)
            },
            'model_info': generation_result.get('model', 'unknown')
        }

        # Сохранение в кэш
        if use_cache:
            self.query_cache[question] = result

        if verbose:
            print(f"✅ Ответ готов! Общее время: {total_time:.2f} сек")
        return result

    # 3. Batch-оценка
    def batch_evaluate(self, test_questions: List[Dict],
                       verbose: bool = False) -> Dict:
        """Пакетная оценка на тестовых вопросах"""

        print(f"🧪 Начало оценки на {len(test_questions)} вопросах...")

        results = []
        correct = 0

        for i, qa in enumerate(test_questions, 1):
            if verbose:
                print(f"\n--- Вопрос {i}/{len(test_questions)} ---")

            # Получаем ответ от RAG
            result = self.query(qa['question'], use_cache=False, verbose=verbose)


            # Задание 6 Напишите код сравнения ответа модели и эталонного
            model_answer = result['answer']
            reference_answer = qa.get('answer', '')

            is_correct = self._evaluate_answer(model_answer, reference_answer)
            if is_correct:
              correct +=1

            results.append({
                'question': qa['question'],
                'predicted': model_answer,
                'expected': reference_answer,
                'is_correct': is_correct
            })

            # Прогресс
            if i % 10 == 0:
                print(f"Обработано: {i}/{len(test_questions)}")

        # Сводная статистика
        accuracy = correct / len(test_questions) if test_questions else 0
        avg_retrieval = np.mean(self.metrics['avg_retrieval_score'])
        avg_time = np.mean(self.metrics['avg_response_time'])

        evaluation_summary = {
            'total_questions': len(test_questions),
            'correct_answers': correct,
            'accuracy': accuracy,
            'avg_retrieval_relevance': avg_retrieval,
            'avg_response_time': avg_time,
            'cache_hit_rate': self.metrics['cache_hits'] / self.metrics['total_queries'],
            'detailed_results': results
        }

        print(f"\n📊 Результаты оценки:")
        print(f"   Точность: {accuracy:.2%}")
        print(f"   Средняя релевантность: {avg_retrieval:.3f}")
        print(f"   Среднее время ответа: {avg_time:.2f} сек")

        return evaluation_summary

    # 4. Методы поддержки
    def _evaluate_answer(self, predicted: str, expected: str) -> bool:
        """Простая оценка правильности ответа"""
        # Нормализация для сравнения
        predicted_lower = predicted.lower().strip()
        expected_lower = expected.lower().strip()

        # Проверка точного совпадения или вхождения
        return (expected_lower in predicted_lower or
                predicted_lower in expected_lower)

    def get_statistics(self) -> Dict:
        """Получение статистики работы системы"""
        return {
            'total_queries': self.metrics['total_queries'],
            'cache_hits': self.metrics['cache_hits'],
            'cache_hit_rate': self.metrics['cache_hits'] / max(self.metrics['total_queries'], 1),
            'avg_retrieval_score': np.mean(self.metrics['avg_retrieval_score']) if self.metrics['avg_retrieval_score'] else 0,
            'avg_response_time': np.mean(self.metrics['avg_response_time']) if self.metrics['avg_response_time'] else 0,
            'total_documents': len(self.vector_store.metadata),
        }


Задание 7

In [ ]:
# Выборка вопросов
questions = [qa_pairs[800]["question"], qa_pairs[0]["question"], qa_pairs[998]["question"], qa_pairs[3]["question"], qa_pairs[600]["question"]]

In [ ]:
# Инициализация RAG системы
rag_system = DeepSeekRAG(config)

# Индексация документов
rag_system.index_documents(documents)

# Тестирование на одном вопросе

# Задание 7 протестируйте вопросы из списка questions с различными настройками DeepSeekRAGConfig (размер чанка, ретривела и тп)
# Опишите, как меняются ответы, точность, скорость. уверенность модели на них, какие характеристики вам кажутся наиболее подходящими.

test_question = questions[0]
result = rag_system.query(test_question, use_cot=False, verbose=True)

print(f"\n🎯 Вопрос: {result['question']}")
print(f"💡 Ответ: {result['answer']}")
print(f"\n📊 Метрики:")
print(f"   - Время поиска: {result['metrics']['retrieval_time']:.3f} сек")
print(f"   - Время генерации: {result['metrics']['generation_time']:.3f} сек")
print(f"   - Средняя релевантность: {result['metrics']['avg_relevance']:.3f}")


🔄 Начало индексации документов...
Обработано: 334 чанков из 85 документов
Генерация эмбеддингов...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Построение FAISS индекса...
Индекс построен: 334 векторов
Индексация завершена: 334 чанков
🔍 Поиск релевантных документов для: 'Chopin composed several songs to lyrics of what language?'
📚 Найдено 5 релевантных контекстов
📚 Relevant_contexts {'chunk_id': '9006904d_part_0', 'text_content': "All of Chopin's compositions include the piano. Most are for solo piano, though he also wrote two piano concertos, a few chamber pieces, and some songs to Polish lyrics", 'meta_info': {'source_doc_id': '9006904d', 'doc_title': 'Frédéric_Chopin', 'position': 0, 'chunks_total': 4, 'length_chars': 168, 'words_count': 29}, 'score': 0.797836422920227, 'relevance': np.float32(0.8989182)}
   Время поиска: 0.06 сек
🤖 Генерация ответа с помощью deepseek-chat
response:  {
    "reasoning": "Source [1] explicitly states that Chopin wrote 'some songs to Polish lyrics'.",
    "answer": "Polish"
}
✅ Ответ готов! Общее время: 2.66 сек

🎯 Вопрос: Chopin composed several songs to lyrics of what language?
💡 Ответ: Poli

In [ ]:
from copy import deepcopy

# Базовая конфигурация
base_config = deepcopy(config)

experiments = [
    {"chunk_size": 100, "retrieval_top_k": 3, "label": "Мелкие чанки, быстрый поиск"},
    {"chunk_size": 300, "retrieval_top_k": 5, "label": "Базовая конфигурация"},
    {"chunk_size": 500, "retrieval_top_k": 10, "label": "Крупные чанки, глубокий поиск"},
]

for exp in experiments:
    print("\n" + "="*80)
    print(f"🧩 Эксперимент: {exp['label']}")
    print("="*80)

    # обновляем конфиг
    exp_config = deepcopy(base_config)
    exp_config.chunk_size = exp["chunk_size"]
    exp_config.retrieval_top_k = exp["retrieval_top_k"]

    # создаем новую систему
    rag_exp = DeepSeekRAG(exp_config)
    rag_exp.index_documents(documents)

    # тестируем один вопрос
    test_question = questions[0]
    result = rag_exp.query(test_question, use_cot=True, verbose=False)

    print(f"🎯 Вопрос: {result['question']}")
    print(f"💡 Ответ: {result['answer']}")
    print(f"⏱ Время: {result['metrics']['total_time']:.2f} сек")
    print(f"📊 Релевантность: {result['metrics']['avg_relevance']:.3f}")


🧩 Эксперимент: Мелкие чанки, быстрый поиск
🔄 Начало индексации документов...
Обработано: 1111 чанков из 85 документов
Генерация эмбеддингов...


Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Построение FAISS индекса...
Индекс построен: 1111 векторов
Индексация завершена: 1111 чанков
response:  {
    "reasoning": "step-by-step reasoning: 1. The question asks about the language of lyrics in songs composed by Chopin. 2. Examining the provided context, Source 3 states: 'some songs to Polish lyrics'. 3. This directly answers the question, confirming the lyrics were in Polish. 4. No other context contradicts or adds to this information.",
    "answer": "Polish"
}
🎯 Вопрос: Chopin composed several songs to lyrics of what language?
💡 Ответ: Polish
⏱ Время: 5.50 сек
📊 Релевантность: 0.869

🧩 Эксперимент: Базовая конфигурация
🔄 Начало индексации документов...
Обработано: 334 чанков из 85 документов
Генерация эмбеддингов...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Построение FAISS индекса...
Индекс построен: 334 векторов
Индексация завершена: 334 чанков
response:  {
    "reasoning": "step-by-step reasoning: 1. The question asks about the language of lyrics for songs composed by Chopin. 2. Source 1 states: 'some songs to Polish lyrics'. 3. Source 2 mentions: 'some of whose verses he set as songs', referring to Adam Mickiewicz, a Polish poet. 4. No other sources provide information about the language of Chopin's song lyrics. 5. The context clearly indicates that Chopin composed songs to Polish lyrics.",
    "answer": "Polish"
}
🎯 Вопрос: Chopin composed several songs to lyrics of what language?
💡 Ответ: Polish
⏱ Время: 5.89 сек
📊 Релевантность: 0.868

🧩 Эксперимент: Крупные чанки, глубокий поиск
🔄 Начало индексации документов...
Обработано: 194 чанков из 85 документов
Генерация эмбеддингов...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Построение FAISS индекса...
Индекс построен: 194 векторов
Индексация завершена: 194 чанков
response:  {
    "reasoning": "Step 1: The question asks about the language of lyrics for songs composed by Chopin. Step 2: Reviewing the context, Источник 1 states: 'some songs to Polish lyrics.' Step 3: This directly answers the question, indicating Chopin composed songs to Polish lyrics.",
    "answer": "Polish"
}
🎯 Вопрос: Chopin composed several songs to lyrics of what language?
💡 Ответ: Polish
⏱ Время: 4.57 сек
📊 Релевантность: 0.843


**С CoT:** ответы становятся более развернутыми, модель поясняет ход рассуждений, еще точность немного выше, если контексты сложные или нужно рассуждение, скорость небыстрая.

**Без CoT:** ответы короткие, быстрые, но иногда модель ошибается, если факты разбросаны по нескольким чанкам. Подходит для быстрых QA, где не нужно объяснение.

**Наблюдения и идеи:**


*   `chunk_size`: увеличивает скорость, снижает точность
*   `retrieval_top_k`: увеличивает полноту, снижает скорость
*   `use_cot=True`: увеличивает точность, увеличивает время
*   `temperature`: увеличвает разнообразие, снижает стабильность



**Итоговый лучше параметры РАГа на ваш взгляд**


*   `chunk_size` = 300 - баланс между скоростью и точностью.
*   `chunk_overlap`= 50 - достаточный контекст на стыках.
*   `retrieval_top_k`= 5 - дает 1-2 релевантных контекста почти всегда.
*   `retrank_top_k`= 3 - отсекает нерелевантные куски.
*   `use_cot=True` - модель дает четкие фактические ответы.
*   `temperature`=0.1 - улучшает осмысленность и объяснения.